In [1]:
!pip install surprise

In [2]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic
import numpy as np

In [3]:
df_cliente = pd.DataFrame({'nomeCliente': ['Ana', 'Claudia', 'Marcos', 'Pedro']})
df_produto = pd.DataFrame({'nomeProduto': ['Cartão de Crédito', 'Conta Corrente', 'Crédito Pessoal', 'Poupança', 'Renda Fixa', 'Renda Variável'], 'quantidadeLikes': [1, 3, 2, 0, 0, 0]})
df_cliente_produto = pd.DataFrame({'nomeCliente': ['Ana', 'Ana', 'Ana', 'Ana', 'Ana', 'Marcos', 'Marcos', 'Marcos', 'Marcos', 'Marcos', 'Pedro', 'Pedro', 'Pedro', 'Pedro', 'Claudia', 'Claudia', 'Claudia'], 'nomeProduto': ['Cartão de Crédito', 'Conta Corrente', 'Crédito Pessoal', 'Poupança', 'Renda Fixa', 'Cartão de Crédito', 'Conta Corrente', 'Poupança', 'Renda Fixa', 'Renda Variável', 'Cartão de Crédito', 'Conta Corrente', 'Poupança', 'Crédito Pessoal', 'Cartão de Crédito', 'Conta Corrente', 'Poupança'], 'valor': [1, 2, 3, 4, 5, 2, 3, 4, 5, 0.6, 3, 4, 5, 7, 4, 5, 6]})


In [4]:
# Filtragem Colaborativa
leitor = Reader(rating_scale=(1, 5))
dados = Dataset.load_from_df(df_cliente_produto[['nomeCliente', 'nomeProduto', 'valor']], leitor)

# Inicializar o modelo KNN
opcoes_sim = {'name': 'cosine', 'user_based': False}
modelo = KNNBasic(sim_options=opcoes_sim)
modelo.fit(dados.build_full_trainset())


Computing the cosine similarity matrix...
Done computing similarity matrix.


In [5]:
# Função para recomendações de Filtragem Colaborativa
def recomendacoes_filtragem_colaborativa(id_usuario, modelo=modelo, df=df_cliente_produto):
    itens_usuario = df[df['nomeCliente'] == id_usuario]['nomeProduto']
    recomendacoes = []
    for item in modelo.trainset.all_items():
        if item not in itens_usuario:
            vizinhos = modelo.get_neighbors(item, k=4)
            recomendacoes.extend(vizinhos)
    return recomendacoes


In [6]:
# Fazer recomendações para a cliente "Marcos"
usuario = 'Marcos'
recomendacoes_cf = recomendacoes_filtragem_colaborativa(usuario)

# Converter a lista de índices em um conjunto para eliminar itens duplicados
recomendacoes_unicas = list(set(recomendacoes_cf))

# Criar um DataFrame com os nomes e quantidades de likes dos produtos recomendados
df_recomendacoes = df_produto.iloc[recomendacoes_unicas]

# Filtrar as linhas que têm a quantidade de likes diferente de zero
df_recomendacoes = df_recomendacoes[df_recomendacoes['quantidadeLikes'] > 0]

# Ordenar os produtos recomendados pela quantidade de likes, de forma decrescente
df_recomendacoes = df_recomendacoes.sort_values(by='quantidadeLikes', ascending=False)

print("\nRecomendações utilizando filtro colaborativo para o cliente:" , usuario)
print("")
print(df_recomendacoes)


Recomendações utilizando filtro colaborativo para o cliente: Marcos

         nomeProduto  quantidadeLikes
1     Conta Corrente                3
2    Crédito Pessoal                2
0  Cartão de Crédito                1
